In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.applications import EfficientNetV2B0
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import io
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from operator import itemgetter
import os
import albumentations as A


In [2]:
def is_kaggle():
    return os.path.exists('/kaggle')

class Config:
    BASE_PATH = '/kaggle/input/isic-2024-challenge/' if is_kaggle() else 'isic-2024-challenge/'
    TRAIN_IMAGE_PATH = 'train-image.hdf5'
    TRAIN_METADATA_PATH = 'train-metadata.csv'
    TEST_IMAGE_PATH = 'test-image.hdf5'
    TEST_METADATA_PATH = 'test-metadata.csv'
    
    # Data processing
    IMAGE_SIZE = (224, 224)
    VALIDATION_SPLIT = 0.15
    RANDOM_STATE = 42
    
    BATCH_SIZE = 64

# Preprocessing

In [4]:
train_hdf5 = h5py.File(Config.BASE_PATH + Config.TRAIN_IMAGE_PATH, 'r')
test_hdf5 = h5py.File(Config.BASE_PATH + Config.TEST_IMAGE_PATH, 'r')

train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)
test_metadata = pd.read_csv(Config.BASE_PATH + Config.TEST_METADATA_PATH)

fnames = train_metadata["isic_id"].tolist()
test_fnames = test_metadata["isic_id"].tolist()

train_target = train_metadata["target"]

split = StratifiedShuffleSplit(n_splits=1, test_size=Config.VALIDATION_SPLIT, random_state=Config.RANDOM_STATE)
for train_index, val_index in split.split(train_metadata, train_target):
    val_fnames = itemgetter(*val_index)(fnames)
    train_fnames = itemgetter(*train_index)(fnames)
    y_train, y_val = train_target.iloc[train_index], train_target.iloc[val_index]

/tmp/ipykernel_34/3660601404.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)


In [6]:
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import albumentations as A

train_transforms = A.Compose([
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        ], p=1.)

def create_image_dataset(fnames, targets, hdf5):
    target_ds = tf.data.Dataset.from_tensor_slices(targets)
    
    def load_image(id):
        image = Image.open(io.BytesIO(np.array(hdf5[id.numpy()])))
        image = np.array(image.resize(Config.IMAGE_SIZE))
        image = (image - image.min()) / (image.max() - image.min())
        return image

    # It doesn't work without this in Kaggle
    def set_shapes(image):
        image.set_shape([*Config.IMAGE_SIZE, 3])
        return image

    # Create a dataset for images
    image_ds = tf.data.Dataset.from_tensor_slices(tf.constant(fnames))
    image_ds = image_ds.map(lambda x: tf.py_function(load_image, [x], tf.float32))
    image_ds = image_ds.map(set_shapes)
    solo_image_ds = tf.data.Dataset.zip((image_ds, target_ds))

    return solo_image_ds

train_solo_image_ds = create_image_dataset(train_fnames, y_train, train_hdf5)
val_solo_image_ds = create_image_dataset(val_fnames, y_val, train_hdf5)

In [7]:
# Create a reduced set of training and validation for the CNN
y_train_reset = y_train.reset_index(drop=True)

# Get the positive instances
positive_indices = np.where(y_train_reset == 1)[0]
train_limit = positive_indices.shape[0]//6
positive_val_indices = positive_indices[:train_limit]
positive_train_indices = positive_indices[train_limit:]
positive_val_count = len(positive_val_indices)
positive_train_count = len(positive_train_indices)

negative_ratio = 200
negative_indices = np.where(y_train_reset == 0)[0]
negative_train_count = negative_ratio * positive_train_count
negative_val_count = negative_ratio * positive_val_count
negative_train_indices = np.random.choice(negative_indices, size=negative_train_count, replace=False)
negative_val_indices = np.random.choice(negative_indices, size=negative_val_count, replace=False)

# Combine positive and selected negative indices
selected_train_indices = np.concatenate([positive_train_indices, negative_train_indices])
selected_val_indices = np.concatenate([positive_val_indices, negative_val_indices])

# Shuffle the indices
np.random.shuffle(selected_train_indices)
np.random.shuffle(selected_val_indices)

# Create the reduced datasets
train_fnames_reduced = [train_fnames[i] for i in selected_train_indices]
y_train_reduced = y_train_reset[selected_train_indices]
val_fnames_reduced = [train_fnames[i] for i in selected_val_indices]
y_val_reduced = y_train_reset[selected_val_indices]

print(f"Total train samples: {len(selected_train_indices)}")
print(f"Positive train samples: {positive_train_count}")
print(f"Negative train samples: {negative_train_count}")
print(f"Total val samples: {len(selected_val_indices)}")
print(f"Positive val samples: {positive_val_count}")
print(f"Negative val samples: {negative_val_count}")

# Now create the dataset with the reduced and balanced data
train_solo_image_reduced_ds = create_image_dataset(train_fnames_reduced, y_train_reduced, train_hdf5)
val_solo_image_reduced_ds = create_image_dataset(val_fnames_reduced, y_val_reduced, train_hdf5)

def augment_image(image, label):
    def apply_augmentation(img):
        img = img.numpy()
        img = train_transforms(image=img)['image']
        return img

    augmented_image = tf.py_function(apply_augmentation, [image], tf.float32)
    augmented_image.set_shape(image.shape)
    return augmented_image, label

# Apply the augmentation to the dataset
train_solo_image_reduced_ds = train_solo_image_reduced_ds.map(augment_image).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_solo_image_reduced_ds = val_solo_image_reduced_ds.batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Total train samples: 56079
Positive train samples: 279
Negative train samples: 55800
Total val samples: 11055
Positive val samples: 55
Negative val samples: 11000


# Auxiliary functions and classes

In [8]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def pauc_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    v_gt = abs(y_true - 1)
    v_pred = 1.0 - y_pred
    min_tpr = 0.80
    max_fpr = 1 - min_tpr
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

class PAUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super(PAUCCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        # Get predictions for validation data
        val_pred = self.model.predict(self.validation_data, verbose=0)
        
        # Extract true labels from validation data
        y_val = np.concatenate([y for x, y in self.validation_data], axis=0)
        
        # Calculate pAUC score
        pauc = pauc_score(y_val, val_pred)
        
        # Optionally, you can add the pAUC score to the logs
        logs['val_pauc'] = pauc

# Image Module

In [92]:
efficientNet = EfficientNetV2B0(weights='imagenet', pooling='avg', include_top=False)

image_input = tf.keras.Input(shape=(*Config.IMAGE_SIZE, 3))
x = efficientNet(image_input)
x = Dense(512, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.4)(x)
x = Dense(128, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)
image_model = tf.keras.Model(inputs=image_input, outputs=x)

In [93]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Create a custom learning rate schedule
def lr_schedule(epoch, lr):
    return lr * 0.7

reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/image_model_epoch_{epoch:02d}_unfreezed.keras',
    save_freq='epoch',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)
pauc_callback = PAUCCallback(val_solo_image_reduced_ds)

callbacks = [model_checkpoint, pauc_callback, reduce_lr]

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) if is_kaggle() else tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)
image_model.compile(optimizer=optimizer, loss='binary_crossentropy')

image_model.fit(
    train_solo_image_reduced_ds,
    epochs=8,
    callbacks=callbacks,
    validation_data=val_solo_image_reduced_ds
)

Epoch 1/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - loss: 0.3760
Epoch 1: saving model to models/image_model_epoch_01_unfreezed.keras
445/445 ━━━━━━━━━━━━━━━━━━━━ 259s 364ms/step - loss: 0.3756 - val_loss: 0.0962 - val_pauc: 0.0594 - learning_rate: 5.0000e-04
Epoch 2/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.0941
Epoch 2: saving model to models/image_model_epoch_02_unfreezed.keras
445/445 ━━━━━━━━━━━━━━━━━━━━ 84s 189ms/step - loss: 0.0941 - val_loss: 0.5474 - val_pauc: 0.0741 - learning_rate: 2.5000e-04
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.0762
Epoch 3: saving model to models/image_model_epoch_03_unfreezed.keras
445/445 ━━━━━━━━━━━━━━━━━━━━ 84s 188ms/step - loss: 0.0762 - val_loss: 0.0743 - val_pauc: 0.1143 - learning_rate: 1.2500e-04
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.0717
Epoch 4: saving model to models/image_model_epoch_04_unfreezed.keras
445/445 ━━━━━━━━━━━━━━━━━━━━ 83s 186ms/step - loss: 0.0717 - val_loss: 0

In [18]:
epoch_model = 8
image_model = tf.keras.models.load_model(f"/kaggle/input/mejores-modelos-isic-2024/models/image_model_epoch_{epoch_model:02d}_unfreezed.keras", compile=False)

In [19]:
tta_transforms = [
    A.RandomRotate90(p=1.0),
    A.Flip(p=1.0),
    A.ShiftScaleRotate(shift_limit=0.1, 
                        scale_limit=0.15, 
                        rotate_limit=60, 
                        p=1.0),
    A.HueSaturationValue(
            hue_shift_limit=0.2, 
            sat_shift_limit=0.2, 
            val_shift_limit=0.2, 
            p=1.0
        ),
    A.RandomBrightnessContrast(
            brightness_limit=(-0.1,0.1), 
            contrast_limit=(-0.1, 0.1), 
            p=1.0
        )
]

def apply_transformation(image, transformation):
    def apply_augmentation(img):
        img = img.numpy()
        img = transformation(image=img)['image']
        return img

    augmented_image = tf.py_function(apply_augmentation, [image], tf.float32)
    augmented_image.set_shape(image.shape)
    return augmented_image

val_tta_preds = []
n = 400

non_transformed_preds = image_model.predict(val_solo_image_ds.batch(128).prefetch(tf.data.AUTOTUNE).take(n), verbose=1)
val_tta_preds.append(non_transformed_preds.flatten())

for transform in tta_transforms:
    augmented_ds = val_solo_image_ds.map(lambda x, y: (apply_transformation(x, transform), y)).batch(128).prefetch(tf.data.AUTOTUNE).take(n)
    preds = image_model.predict(augmented_ds, verbose=1)
    val_tta_preds.append(preds.flatten())

# Average the predictions
val_tta_preds_mean = np.mean(val_tta_preds, axis=0)
val_tta_pauc = pauc_score(y_val[:128*n], val_tta_preds_mean)
print(f"Validation pAUC score with TTA: {val_tta_pauc:.4f}")

400/400 ━━━━━━━━━━━━━━━━━━━━ 111s 262ms/step
400/400 ━━━━━━━━━━━━━━━━━━━━ 152s 380ms/step
Validation pAUC score with TTA: 0.1340


In [23]:
select = [0]
x = np.mean(np.array(val_tta_preds)[select], axis=0)
pauc_score(y_val[:128*n], x)

0.137797375699295